In [ ]:
from function import ViolationDetector  # Import your class from the first file
import os
import cv2
import torch
import time
var = ViolationDetector()
img = []
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
for j,i in enumerate(os.listdir("/home/annone/ai/images/urination")):
    if j == 80:
        break
    if i.endswith(".png"):  # Check if the file is a JPG image
        img_path = os.path.join("/home/annone/ai/images/urination", i)  # Create the full path
        img.append(cv2.imread(img_path))

model_path = '/home/annone/ai/models/pee_spit.pth'
t1 = time.time()
results = var.process_image(img,model_path , "uri_000", "000000", "00000")
t2 = time.time()
print(t2-t1)
print(results)

In [ ]:
import cv2
import multiprocessing as mp

# Define paths to the video files
video_paths = [
    '/home/annone/ai/data/wrongway.mp4', '/home/annone/ai/data/wrongway.mp4', '/home/annone/ai/data/wrongway.mp4', '/home/annone/ai/data/wrongway.mp4', '/home/annone/ai/data/wrongway.mp4',
    '/home/annone/ai/data/wrongway.mp4'
]

# Function to read frames from a video file and display them
def display_video(video_path, window_name):
    cap = cv2.VideoCapture(video_path)
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print(f"Video {window_name} has ended.")
            break
        
        # Display the frame in a window with the given name
        cv2.imshow(window_name, frame)

        # Exit if 'q' key is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyWindow(window_name)

# Function to start processes for each video
def main():
    processes = []
    
    # Launch a separate process for each video
    for i, video_path in enumerate(video_paths):
        window_name = f"Video {i + 1}"
        process = mp.Process(target=display_video, args=(video_path, window_name))
        processes.append(process)
        process.start()
    
    # Join all processes to wait until they are finished
    for process in processes:
        process.join()

if __name__ == "__main__":
    main()
    cv2.destroyAllWindows()


In [ ]:
import cv2
import multiprocessing

# Function to read frames from a single video
def process_video(video_path, video_index):
    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    
    # Read frames until video ends or process is manually terminated
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        # Simulate frame processing by printing frame count
        print(f"Video {video_index}: Processing frame {frame_count}")
        frame_count += 1
        cv2.imshow(f"{video_index}",frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    print(f"Video {video_index}: Finished processing")

if __name__ == "__main__":
    # Define paths to your video files
    video_paths = [
    'rtmp://122.200.18.78:80/live',
    'rtmp://122.200.18.78:80/live'
]
    # Create a pool of 16 processes (one per video)
    with multiprocessing.Pool(processes=16) as pool:
        # Map each video file to the process_video function
        pool.starmap(process_video, [(video_path, idx+1) for idx, video_path in enumerate(video_paths)])


In [ ]:
import cv2
import random
def process_frame(frame):
    # Add your object detection logic or processing here
    # For demonstration, let's just convert the frame to grayscale
    processed_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    return processed_frame

def main():
    video_feeds = [
        'rtmp://122.200.18.78:80/live',
        'rtmp://122.200.18.78:80/live','rtmp://122.200.18.78:80/live'
    ]

    caps = [cv2.VideoCapture(video) for video in video_feeds]

    while True:
        frames = []
        for cap in caps:
            ret, frame = cap.read()
            frames.append(frame)
            if not ret:
                print("End of video or cannot read frame.")
                continue
        print(len(frames))
        for i, frame in enumerate(frames):
            print(i)
            print(frame)
            frame = cv2.resize(frame, (640,480))
            # processed_frame = process_frame(frame)
            cv2.imshow(f'{i}', frame)
        frames = []
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        
        # Check if all video captures are done
        # if all(not cap.isOpened() or cap.get(cv2.CAP_PROP_POS_FRAMES) >= cap.get(cv2.CAP_PROP_FRAME_COUNT) for cap in caps):
        #     break

    # Release all video capture objects
    for cap in caps:
        cap.release()
    cv2.destroyAllWindows()

if __name__ == '__main__':
    main()


In [ ]:
import cv2
import numpy as np
import torch
from ultralytics import YOLO
import time

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = YOLO(f"/home/annone/ai/models/yolov8n.pt")
model.to(device)
def process_frames(frames):
    resized_frames = [
        cv2.resize(frame, (640 // 32 * 32, 480 // 32 * 32))
        for frame in frames
    ]
    frames_tensor = (
        torch.from_numpy(np.stack(resized_frames))
        .permute(0, 3, 1, 2)
        .float()
        .to(device)
        / 255.0
    )

    with torch.no_grad():
        batch_results = model(frames_tensor, device=device)

    return batch_results

def track_frames(batch_results):
    time.sleep(1)
    return batch_results

def main():
    video_feeds = [
        '/home/annone/ai/data/wrongway.mp4',
        '/home/annone/ai/data/wrongway.mp4',
        # '/home/annone/ai/data/wrongway.mp4',
        # '/home/annone/ai/data/wrongway.mp4',
        # '/home/annone/ai/data/wrongway.mp4',
    ]

    caps = [cv2.VideoCapture(video) for video in video_feeds]

    while True:
        frames = []

        # Read frames from all video feeds
        for cap in caps:
            ret, frame = cap.read()
            if not ret:
                print("End of video or cannot read frame.")
                frames.append(None)  # Append None if no frame is captured
            else:
                # Resize the frame
                frame = cv2.resize(frame, (640, 480))
                frames.append(frame)

        # Check if all video captures are done
        if all(frame is None for frame in frames):
            break
        
        # Process all captured frames together
        processed_frames = process_frames(frames)
        # tracked_frames = track_frames(processed_frames)
        # Display the resulting frames
        for i, processed_frame in enumerate(processed_frames):
            if processed_frame is not None:
                cv2.imshow(f'Camera Feed {i + 1}', frame)

        # Break the loop on 'q' key press
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release all video capture objects
    for cap in caps:
        cap.release()
    cv2.destroyAllWindows()

if __name__ == '__main__':
    main()


In [3]:
import multiprocessing
import subprocess

def run_script1():
    subprocess.run(['python3', '/home/annone/ai/backend/stream/custom_inference.py'])

def run_script2():
    subprocess.run(['python3', '/home/annone/ai/backend/stream/custom_inference.py'])

def run_script3():
    subprocess.run(['python3', '/home/annone/ai/backend/stream/custom_inference.py'])

def run_script4():
    subprocess.run(['python3', '/home/annone/ai/backend/stream/custom_inference.py'])

if __name__ == '__main__':
    # Create two processes for each script
    p1 = multiprocessing.Process(target=run_script1)
    p2 = multiprocessing.Process(target=run_script2)
    p3 = multiprocessing.Process(target=run_script3)
    p4 = multiprocessing.Process(target=run_script4)

    # Start the processes
    p1.start()
    p2.start()
    p3.start()
    p4.start()

    # Wait for both processes to finish
    p1.join()
    p2.join()
    p3.join()
    p4.join()

    print("Both scripts have finished execution.")


Using CPU. Note: This module is much faster with a GPU.
Using CPU. Note: This module is much faster with a GPU.
Using CPU. Note: This module is much faster with a GPU.
Using CPU. Note: This module is much faster with a GPU.


{'dbname': 'logs', 'user': 'postgres', 'password': 'argus123', 'host': '34.47.148.81', 'port': '8080'}
{'dbname': 'logs', 'user': 'postgres', 'password': 'argus123', 'host': '34.47.148.81', 'port': '8080'}
{'dbname': 'logs', 'user': 'postgres', 'password': 'argus123', 'host': '34.47.148.81', 'port': '8080'}
{'dbname': 'logs', 'user': 'postgres', 'password': 'argus123', 'host': '34.47.148.81', 'port': '8080'}
/home/annone/ai/data/wrongway.mp4
[[[417, 476.25], [383, 144.25], [380, 96.25], [408, 90.25], [446, 90.25], [484, 103.25], [515, 118.25], [562, 166.25], [595, 196.25], [640, 233.25], [638, 479.25]], [[1, 478.25], [302, 479.25], [309, 366.25], [294, 99.25], [244, 126.25], [162, 159.25], [91, 231.25], [0, 293.25]]]
/home/annone/ai/data/wrongway.mp4
[[[417, 476.25], [383, 144.25], [380, 96.25], [408, 90.25], [446, 90.25], [484, 103.25], [515, 118.25], [562, 166.25], [595, 196.25], [640, 233.25], [638, 479.25]], [[1, 478.25], [302, 479.25], [309, 366.25], [294, 99.25], [244, 126.25], [

In [ ]:
import cv2

# Define the RTMP stream URLs
stream_1_url = 'rtmp://122.200.18.78:80/live/one'
stream_2_url = 'rtmp://122.200.18.78:80/live/two'
stream_3_url = 'rtmp://122.200.18.78:80/live/three'

# Open video capture for each stream
cap1 = cv2.VideoCapture(stream_1_url)
cap2 = cv2.VideoCapture(stream_2_url)
cap3 = cv2.VideoCapture(stream_3_url)

# Check if the streams opened successfully
if not (cap1.isOpened() and cap2.isOpened() and cap3.isOpened()):
    print("Error: Could not open one or more streams.")
    exit()

while True:
    # Read frames from each stream
    ret1, frame1 = cap1.read()
    ret2, frame2 = cap2.read()
    ret3, frame3 = cap3.read()

    # Check if frames are read successfully
    if ret1:
        cv2.imshow('Stream 1', frame1)
    if ret2:
        cv2.imshow('Stream 2', frame2)
    if ret3:
        cv2.imshow('Stream 3', frame3)

    # Press 'q' to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture objects and close all windows
cap1.release()
cap2.release()
cap3.release()
cv2.destroyAllWindows()


In [ ]:
import cv2
import mediapipe as mp

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

hands = mp_hands.Hands()

cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        continue

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

    cv2.imshow("Hand Gesture Recognition", frame)
    if cv2.waitKey(10) == 27:  # Press 'Esc' to exit
        break

cap.release()
cv2.destroyAllWindows()

2024-11-18 15:12:10.550271: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731922930.562805  176300 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731922930.566715  176300 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-18 15:12:10.581567: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
I0000 00:00:1731922932.492921  176300 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1731